In [7]:
import pickle
import sagemaker
import boto3
import os
import fxcmpy
import mxnet as mx
import xgboost as xgb
file = open('xgboost_model', 'rb')
xgb_model = pickle.loads(file.read())
unzipped_model = 'xgboost_model'
 
#model_xgb = xgb.Booster()
#model_xgb.load_model(unzipped_model)

In [8]:
TOKEN = 'ADD_TOKEN_HERE'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error')

In [9]:
new_data = con.get_candles('EUR/USD', period='H4', number=8000)

In [10]:
new_data.to_csv('EUR_USD_H4_10L.csv')

In [11]:
def create_lags_testing(dataset, lags_num):
    data = dataset.drop(columns = ['askopen','askclose', 'askhigh', 'asklow'] )
    cols = []
    col = ''
    #cols.append('tickqty')
   #print(data.head())
    for lag in range(1, lags_num):
       # col = 'bidopen_lag_%s' % lag
        #cols.append(col)
        #data[col] = data['bidopen'].shift(lag)
        ##################################################
        col = 'bidclose_lag_%s' % lag
        cols.append(col)
        data[col] = data['bidclose'].shift(lag)
        ##################################################
        col = 'tick_quant_lag_%s' % lag
        cols.append(col)
        data[col] = data['tickqty'].shift(lag)
        
    return data, cols

In [12]:
alt,columns = create_lags_testing(new_data,11)
alt = alt.dropna()

In [13]:
alt.to_csv('EUR_USD_H4_10L.csv')

In [ ]:
X = alt.iloc[:, 5:]
y = alt.iloc[:, 0:2]
X_train = X[:len(X)-30]
X_test = X[len(X)-30:]
y_train = y[:len(y)-30]
y_test = y[len(y)-30:]

In [ ]:
alt.tail()

In [ ]:
X.head()

In [ ]:
import xgboost as xgb
data_input = xgb.DMatrix(alt.iloc[:, 5:].values)
predictions = xgb_model.predict(data_input)

In [ ]:
predictions